In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils.io_utils import *
from utils.opensim_kinematics_utils import *
from utils.data_processing_utils import *
from utils.analysis_utils import *

In [2]:
# define variables

MODEL = "gait2354_simbody.osim"
MARKERSET_FILE = "example_markerset_files/HMB2_MODEL_LUCILLE_markers.xml" # in this file the missing marker should NOT be defined
# list of the names of the markers in MARKERSET_FILE
MARKERS = ["C7", "T10", "STRN", "LASI", "RASI", "LPSI", "RPSI",
               "LTHI", "LKNE", "LMEK", "LANK", "LMM", "LHEE", "LTOE", "LMT2", "LMT5",
			   "RTHI", "RKNE", "RMEK", "RANK", "RMM", "RHEE", "RTOE", "RMT2", "RMT5",
               "RStatic1", "RStatic2", "RStatic3", "RStatic4", "RStatic5", "RStatic6", "RStatic7",
               "Rrear1", "Rrear2",
               "LStatic1", "LStatic2", "LStatic3", "LStatic4", "LStatic5",
			   "Lrear1", "Lrear2", "LMIMU",
               "pelvis1", "pelvis2", "pelvis3", "pelvis4"
               ]
C3D_CALIBRATION_FILE = "example_c3d_files/GCAP_S_03_TPose.c3d"
TEMPLATE_SCALING_SETUP_FILE = "template_scaling_setup.xml"

RESULTS_FOLDER = "results_add_marker"

# MISSING MARKER
C3D_FILE_TO_ADD_MARKER = "example_c3d_files/GCAP_S_03_TPose.c3d" # in this case the marker is missing in the calibration file but could be any other file
NAME = "RStatic8"
COORDINATES = (0.0412685, -0.0218856, 0.0594629) # coordinates of the missing marker in the referential of the corresponding body part (has to be determined manually in OpenSim)
BODY_PART = "toes_r"
UPDATED_MARKERSET = os.path.join(RESULTS_FOLDER, f"HBM2_MODEL_markers_Lucille_updated.xml") # define path where the updated markerset .xml file will be saved


PREFIX = "add_marker" # needed for naming conventions

In [3]:
os.makedirs(RESULTS_FOLDER, exist_ok=True)  

In [4]:
# STEP 1: SCALE MODEL WITH CALIBRATION FILE, BASED ON ALL MARKERS EXCEPT THE MISSING ONE
scaling(PREFIX,
        RESULTS_FOLDER,
        MARKERS,
        TEMPLATE_SCALING_SETUP_FILE,
        MARKERSET_FILE,
        c3d_calib_file = C3D_CALIBRATION_FILE)

path_to_scaled_model = os.path.join(RESULTS_FOLDER, PREFIX + "_scaled_model_markers.osim")


# STEP 2: INVERSE KINEMATICS WITH .c3d FILE THAT NEEDS TO BE UPDATED (BASED ON MARKERSET WITHOUT MISSING MARKER) TO GET CORRESPONDING .mot file
# (this mot file gives the kinematics of the model while performing the tasks
# and will allow to perform point kineamtics to retrieve the coordinates of the markerset with added missing marker)
(start_time, stop_time) = inverse_kinematics(PREFIX,
                   "GCAP_S_03_TPose",
                   MARKERS,
                   MARKERSET_FILE,
                   path_to_scaled_model,
                   RESULTS_FOLDER,
                   c3d_dynamic_file=C3D_FILE_TO_ADD_MARKER)

path_to_motion_file = os.path.join(RESULTS_FOLDER, PREFIX+"_GCAP_S_03_TPose_motion.mot")

# STEP 3: UPDATE MARKERSET .xml FILE BY ADDING MISSING MARKER AND ITS COORDINATES TO THE EXISTING MARKERSET FILE
add_marker_to_opensim_file(
    file_path=MARKERSET_FILE,
    marker_name=NAME,
    coordinates=COORDINATES,
    body_part=BODY_PART,
    output_file=UPDATED_MARKERSET)

# STEP 4: POINT KINEMATICS WITH UPDATED MARKERSET ON .mot FILE RESULTING FROM THE PREVIOUS INVERSE KINEMATICS STEP
point_kinematics ("updated_markerset_"+PREFIX,
                  "GCAP_S_03_TPose",
                  UPDATED_MARKERSET,
                  RESULTS_FOLDER,
                  path_to_scaled_model,
                  path_to_motion_file,
                  start_time=start_time,
                  end_time=stop_time)

path_to_updated_trc_file = os.path.join(RESULTS_FOLDER, "updated_markerset_"+PREFIX+"_GCAP_S_03_TPose.trc")

# STEP 5: CONVERT .trc FILE RESULTING FROM PREVIOUS POINTKIN STEP INTO .c3d FILE
convert_trc_to_c3d(path_to_updated_trc_file, source_c3d_path=C3D_FILE_TO_ADD_MARKER)

1900
0 100
✅ Conversion terminée : results_add_marker\add_marker_calib_static.trc
1900
0 1900
✅ Conversion terminée : results_add_marker\add_marker_GCAP_S_03_TPose.trc
1 marker(s) added and saved to 'results_add_marker\HBM2_MODEL_markers_Lucille_updated.xml' with proper formatting.
TRC file saved: results_add_marker\updated_markerset_add_marker_GCAP_S_03_TPose.trc
🔄 Ajout des données de force depuis : example_c3d_files/GCAP_S_03_TPose.c3d
💾 Écriture du fichier C3D : results_add_marker\updated_markerset_add_marker_GCAP_S_03_TPose.c3d
✅ Fichier C3D sauvegardé : results_add_marker\updated_markerset_add_marker_GCAP_S_03_TPose.c3d
